In [12]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras import models
from numpy import ndarray

In [13]:
checklist_dict = {
    1101: "Do you read frequently?",
    1102: "Do you maintain good memory?",
    1103: "Do you keep up with the news?",
    1104: "Do you engage in mentally stimulating activities?",
    1105: "Do you read books often?",
    1201: "Have you experienced something joyful?",
    1202: "Do you adapt well to change?",
    1203: "Do you manage stress well?",
    1204: "Do you live a happy life?",
    1205: "Are you at peace?",
    1206: "Do you often feel happy?",
    1301: "Do you dress warmly?",
    1302: "Are you good at household chores?",
    1303: "Do you use transportation well?",
    1304: "Do you live safely?",
    1305: "Are you able to ask for help when needed?",
    1306: "Do you find using a smartphone easy?",
    1307: "Do you live a clean life?",
    1401: "Do you feel physically healthy?",
    1402: "Are you physically strong?",
    1403: "Are you flexible?",
    1404: "Do you have good balance?",
    1405: "Do you exercise regularly?",
    1406: "Do you have awareness of your physical health?",
    1407: "Do you manage any discomforts in your body well?",
    1408: "Do you have good stamina?",
    1501: "Do you eat well?",
    1502: "Do you take supplements/medicine regularly?",
    1503: "Do you eat a nutritious diet?",
    1504: "Do you drink enough water?",
    1505: "Do you have a regular meal schedule?",
    1506: "Do you eat a variety of foods?",
    1601: "Do you get enough sleep?",
    1602: "Do you sleep deeply?",
    1603: "Do you relax during your leisure time?",
    1604: "Do you wake up refreshed in the morning?",
    1605: "Do you move around comfortably?",
    1606: "Do you get enough rest?",
    1701: "Do you regularly communicate with your family?",
    1702: "Do you communicate with your friends?",
    1703: "Do you attend regular gatherings?",
    1704: "Do you communicate comfortably?",
    1705: "Do you use a smartphone or computer for social activities?",
    1706: "Are you currently satisfied with your social activities?"
}
mission_dict = {
    2101: "Read paper news",
    2102: "Solve a puzzle",
    2103: "Solve Sudoku",
    2104: "Read a book",
    2105: "Write a poem",
    2201: "Laugh a lot",
    2202: "Listen to music",
    2203: "Meditate",
    2204: "Watch a movie",
    2205: "Plan a trip",
    2206: "People watch",
    2301: "Grow a plant",
    2302: "Do household chores",
    2303: "Air out the house",
    2304: "Do a deep cleaning",
    2305: "Take a half-bath",
    2306: "Use a comfort app",
    2307: "Take care of appearance",
    2401: "Take a 30-minute or longer walk regularly",
    2402: "Do 30 minutes or more of strength training regularly",
    2403: "Stretch",
    2404: "Do balance exercises",
    2405: "Manage uncomfortable physical parts",
    2406: "Google for any illnesses you are experiencing",
    2407: "Climb stairs",
    2501: "Go to a new restaurant",
    2502: "Prepare three meals a day",
    2503: "Make a variety of menus",
    2504: "Drink 10 cups of water",
    2505: "Eat at a set mealtime",
    2506: "Cook food",
    2601: "Get a lot of sleep",
    2602: "Create a good sleep environment",
    2603: "Rest comfortably",
    2604: "Sleep and wake up at set times",
    2701: "Contact family",
    2702: "Contact acquaintances",
    2703: "Go to a gathering",
    2704: "Try to talk to people you meet",
    2705: "Participate in social activities on the internet with your smartphone",
    2706: "Join a new group",
    2707: "Go to church"
}

In [14]:
shap_values = np.array([-0.0049126 , -0.05862321, -0.02260871, -0.01699826, -0.00154646, -0.02258623, -0.05640625])

adjusted_shap_values = abs(shap_values)
adjusted_shap_values /= max(adjusted_shap_values)
adjusted_shap_values

array([0.08379957, 1.        , 0.38566141, 0.28995785, 0.02637965,
       0.38527795, 0.9621829 ])

    1 out of 3 times, mission is readomly chosen, and for other cases recommendation system collect score data from user and multiply with SHAP value, give weigth on week part of user and key feature that caouse lonely death. 

    Checklist recommendation is fully random, since we wanted to track every feature at the same time.

In [15]:
def recommend_mission(mission_dict: dict,
                      score: ndarray,
                      shap_values: ndarray = adjusted_shap_values) -> int:
    
    
    if random.randint(0, 2) == 1:
        category = random.randint(1, 7)
        code = random.choice([x for x in mission_dict if x > 2000 + category*100 and x < 2000 + (category + 1)*100])

        return code

    score = list(np.multiply(score, shap_values))
    for i, v in enumerate(score):
        score[i] = (i + 1, v)

    sorted_score = sorted(score, key=lambda tup: tup[1])
    category = sorted_score[random.randint(0, 2)][0]

    code = random.choice([x for x in mission_dict if x > 2000 + category*100 and x < 2000 + (category + 1)*100])

    return code


def recommend_checklist(checklist_list: list) -> int:
    category = random.randint(1, 7)

    code = random.choice([x for x in checklist_list if x > 1000 + category*100 and x < 1000 +  (category + 1)*100])

    return code

In [16]:
# Generates arbitrary user(Alice)'s average score

Alice_score = np.random.randn(7)
Alice_score /= max(Alice_score)

print("recommended mission:    ", mission_dict[recommend_mission(mission_dict, Alice_score)])

recommended mission:     Laugh a lot


In [17]:
# Load our model and predict likelihood of lonely death

model = models.load_model("wiro_DL_model.h5", compile= False)
model.compile(optimizer=tf.keras.optimizers.Adam(0.002), loss='mse', metrics=['mae'])
likelihood = model.predict(pd.DataFrame(Alice_score.reshape((1, -1))))

print("likelihood of lonelydeath:   ",  likelihood[(0, 0)])

1/1 [==============================] - 0s 72ms/step
likelihood of lonelydeath:    11.893965
